This is a notebook that 
- Create 10 different test and training set combination
- Directly trains neural network on era5 data 
- Test the average performance by recall and precision tested on 10 testsets. 

Era 5 data is labeled as True or False according to the length of blockings. By changing the value of T in the filename  Ysparse=np.load("/scratch/hz1994/blocking/data_era5/test_labels_1940-2022_T%d.npy"%T) Now the T=5,7,9 which captures longer and longer blocking maintenance.   

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import struct

import cartopy
from cartopy import crs as ccrs
import matplotlib 
from matplotlib import pyplot as plt
import os
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import pandas as pd
import matplotlib.path as mpath
from matplotlib.colors import TwoSlopeNorm 
from sklearn.model_selection import train_test_split
from scipy.fft import fft, ifft
from scipy import signal
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pickle
import sys
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import importlib.util
import pathlib
import MM_util_AI
spec = importlib.util.spec_from_file_location("MM_dataprepare", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_dataprepare.py")
MM_dataprepare = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_dataprepare
spec.loader.exec_module(MM_dataprepare)
spec = importlib.util.spec_from_file_location("MM_utilblocking", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_utilblocking.py")
MM_utilblocking = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_utilblocking
spec.loader.exec_module(MM_utilblocking)
from sklearn.metrics import confusion_matrix,recall_score,precision_score
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("Reduced_dim_variables"):
            rd_path=ln.strip().split('\t')[1]
        if ln.startswith("TMindex_filepath"):
            TMindex_path=ln.strip().split('\t')[1]   
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]   
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1]            
            
print(rd_path)
print(TMindex_path)
print(dim_path)
print(train_path)
print(models_path)
print(fig_path)

def plot_bunch_timeseqdist(ax, matrix,  figtype="shades", number_lines=None, label=None,color=None,linestyle='-',alpha=1):
    #colomns are indicies, rows are time
    xt=np.arange(matrix.shape[1]) 
    if figtype=="shades":
        mean=np.nanmean(matrix,axis=0)
        std=np.nanstd(matrix,axis=0)
        p = ax.plot(xt,mean ,label=label,color=color,alpha=alpha,linestyle=linestyle)
        color=p[0].get_color()
        ax.plot(xt,mean-std,color=color,alpha=0. )
        ax.plot(xt,mean+std,color=color,alpha=0. )
        ax.fill_between(xt, mean-std, mean+std,color=color,alpha=0.1)
    elif figtype=="lines":
        p = ax.plot(xt,matrix.T ,color=color, label=label,alpha=alpha)
    elif figtype=="no":
        mean=np.nanmean(matrix,axis=0)
        std=np.nanstd(matrix,axis=0)
        p = ax.plot(xt,mean ,label=label,color=color,alpha=alpha,linestyle=linestyle)
    else:
        print("Please specify the figtype (shades or lines).")
        

# Prepare the dataset of training and output properties


In [ ]:
for T in [5,7,9]:
    Ysparse=np.load("/scratch/hz1994/blocking/data_era5/test_labels_1940-2022_T%d.npy"%T)  
    print('positive data=',Ysparse.sum(),'negative data=',Ysparse.size-Ysparse.sum())

In [ ]:
M=150 
T=7

X=np.load("/scratch/hz1994/blocking/data_era5/test_data_1940-2022.npy")
Ysparse=np.load("/scratch/hz1994/blocking/data_era5/test_labels_1940-2022_T%d.npy"%T)  
Y=np.zeros((Ysparse.size,2)).astype(bool)
Y[:,1][Ysparse==1]=True  #blocking
Y[:,0][Ysparse==0]=True
pos=Y[:,1].sum()
total=Y[:,1].size
neg=total-pos
print('positive data=',pos,'negative data=',neg)
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight


# Training and testing
Devide the dataset into num equal parts, and create 10 different training/test dataset combination. In this way we can statistically evaluate the average performance.

In [ ]:
num_total=X.shape[0]
seed=10
ordered_ind=np.arange(num_total)
np.random.seed(seed)
np.random.shuffle(ordered_ind)
num=10
shuffled_ind=np.random.shuffle(ordered_ind)
test_data_list=(np.array_split(ordered_ind, num))
train_data_list=[]
for i in range(num):
    train_data_list.append( np.array( [j  for j in ordered_ind if j not in test_data_list[i]]   )  )

In [ ]:
#print each time how many positive events are in the test set.
for i in range(num):
    print(Ysparse[test_data_list[i]].sum())

In [1]:
EPOCH=10
cnnsize="normal"
for random_seed in range(10):
    #specify the savepath of the training history and the checkpoints
    print("randomseed=",random_seed)
    
    history_list=[]
    for i in range(num):
        path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_%ddaysblocking/%s/"%(T,cnnsize)\
            +"random_initCNN_%d/%d/"%(random_seed,i)
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        
        test_ind=test_data_list[i]
        train_ind=train_data_list[i]
        train_data=X[train_ind]
        train_labels=Y[train_ind]
        test_data=X[test_ind]
        test_labels=Y[test_ind]
        
        if cnnsize=="smaller":
            model = MM_util_AI.make_s_model((18,90,3))
        elif cnnsize=="smaller_smaller":
            model = MM_util_AI.make_ss_model((18,90,3))
        elif cnnsize=="normal":
            model = MM_util_AI.make_model((18,90,3))

        checkpoint_path = path+"cp-{epoch:04d}.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)
        model.save_weights(checkpoint_path.format(epoch=0))
    
        model_callbacks = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path, 
            verbose=1, 
            save_weights_only=True,
            save_freq="epoch")
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      metrics=[tf.keras.metrics.Precision(class_id=1,thresholds=0),tf.keras.metrics.Recall(class_id=1,\
                                                                                             thresholds=0) ])

        history = model.fit(train_data, train_labels, epochs=EPOCH, 
                            callbacks=[model_callbacks],
                            validation_data=(test_data, test_labels),
                            class_weight=class_weight)
        history_list.append(history.history)
    with open(path+'/trainHistoryDict_list', 'wb') as file_pi:
        pickle.dump(history_list, file_pi)
            #recall divides true_positives by the sum of true_positives and false_negatives.

Epoch 5/10
5/8 [=================>............] - ETA: 0s - loss: 0.4815 - precision_114: 0.4286 - recall_114: 0.8182
Epoch 5: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/4/cp-0005.ckpt
8/8 [==============================] - 0s 25ms/step - loss: 0.5018 - precision_114: 0.3529 - recall_114: 0.7059 - val_loss: 0.4766 - val_precision_114: 0.0000e+00 - val_recall_114: 0.0000e+00
Epoch 6/10
5/8 [=================>............] - ETA: 0s - loss: 0.4074 - precision_114: 0.1864 - recall_114: 1.0000
Epoch 6: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/4/cp-0006.ckpt
8/8 [==============================] - 0s 23ms/step - loss: 0.4244 - precision_114: 0.2000 - recall_114: 0.8824 - val_loss: 0.5064 - val_precision_114: 0.0000e+00 - val_recall_114: 0.0000e+00
Epoch 7/10
5/8 [=================>............] - ETA: 0s - loss: 0.2947 - preci

Epoch 3/10
5/8 [=================>............] - ETA: 0s - loss: 0.5342 - precision_116: 0.1719 - recall_116: 1.0000
Epoch 3: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/6/cp-0003.ckpt
8/8 [==============================] - 0s 25ms/step - loss: 0.4971 - precision_116: 0.1765 - recall_116: 0.9375 - val_loss: 0.4115 - val_precision_116: 0.2000 - val_recall_116: 0.5000
Epoch 4/10
5/8 [=================>............] - ETA: 0s - loss: 0.3187 - precision_116: 0.1667 - recall_116: 0.5000        
Epoch 4: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/6/cp-0004.ckpt
8/8 [==============================] - 0s 26ms/step - loss: 0.4189 - precision_116: 0.2245 - recall_116: 0.6875 - val_loss: 1.1191 - val_precision_116: 0.0625 - val_recall_116: 0.5000
Epoch 5/10
5/8 [=================>............] - ETA: 0s - loss: 0.4579 - precision_116

Epoch 1/10
6/8 [=====================>........] - ETA: 0s - loss: 0.6822 - precision_118: 0.0476 - recall_118: 0.1667
Epoch 1: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/8/cp-0001.ckpt
8/8 [==============================] - 1s 53ms/step - loss: 0.6479 - precision_118: 0.0423 - recall_118: 0.2143 - val_loss: 0.6109 - val_precision_118: 0.2727 - val_recall_118: 0.7500
Epoch 2/10
5/8 [=================>............] - ETA: 0s - loss: 0.5130 - precision_118: 0.2500 - recall_118: 0.1429        
Epoch 2: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/8/cp-0002.ckpt
8/8 [==============================] - 0s 25ms/step - loss: 0.5726 - precision_118: 0.4000 - recall_118: 0.2857 - val_loss: 0.5695 - val_precision_118: 0.2500 - val_recall_118: 0.7500
Epoch 3/10
5/8 [=================>............] - ETA: 0s - loss: 0.4825 - precision_118

Epoch 9/10
5/8 [=================>............] - ETA: 0s - loss: 0.1132 - precision_119: 0.5500 - recall_119: 1.0000
Epoch 9: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/9/cp-0009.ckpt
8/8 [==============================] - 0s 24ms/step - loss: 0.0974 - precision_119: 0.5926 - recall_119: 1.0000 - val_loss: 0.4140 - val_precision_119: 0.0000e+00 - val_recall_119: 0.0000e+00
Epoch 10/10
5/8 [=================>............] - ETA: 0s - loss: 0.0912 - precision_119: 0.9231 - recall_119: 1.0000
Epoch 10: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_1/9/cp-0010.ckpt
8/8 [==============================] - 0s 25ms/step - loss: 0.1096 - precision_119: 0.6667 - recall_119: 1.0000 - val_loss: 0.8329 - val_precision_119: 0.1429 - val_recall_119: 0.5000
randomseed= 2
Epoch 1/10
6/8 [=====================>........] - ETA: 0s - loss: 0.7623

Epoch 7/10
6/8 [=====================>........] - ETA: 0s - loss: 0.3065 - precision_121: 0.6875 - recall_121: 0.8462        
Epoch 7: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_2/1/cp-0007.ckpt
8/8 [==============================] - 0s 23ms/step - loss: 0.3058 - precision_121: 0.5385 - recall_121: 0.8750 - val_loss: 0.3291 - val_precision_121: 0.1667 - val_recall_121: 0.5000
Epoch 8/10
5/8 [=================>............] - ETA: 0s - loss: 0.2464 - precision_121: 0.3421 - recall_121: 1.0000
Epoch 8: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_2/1/cp-0008.ckpt
8/8 [==============================] - 0s 24ms/step - loss: 0.2150 - precision_121: 0.3404 - recall_121: 1.0000 - val_loss: 0.2428 - val_precision_121: 0.5000 - val_recall_121: 0.5000
Epoch 9/10
6/8 [=====================>........] - ETA: 0s - loss: 0.1587 - precision_121

...

Epoch 9/10
6/8 [=====================>........] - ETA: 0s - loss: 0.2762 - precision_191: 0.2069 - recall_191: 1.0000        
Epoch 9: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/1/cp-0009.ckpt
8/8 [==============================] - 0s 21ms/step - loss: 0.2536 - precision_191: 0.2388 - recall_191: 1.0000 - val_loss: 0.2254 - val_precision_191: 0.4000 - val_recall_191: 1.0000
Epoch 10/10
5/8 [=================>............] - ETA: 0s - loss: 0.2142 - precision_191: 0.6000 - recall_191: 0.9000
Epoch 10: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/1/cp-0010.ckpt
8/8 [==============================] - 0s 22ms/step - loss: 0.2015 - precision_191: 0.6364 - recall_191: 0.8750 - val_loss: 0.3048 - val_precision_191: 0.3333 - val_recall_191: 1.0000
Epoch 1/10
6/8 [=====================>........] - ETA: 0s - loss: 0.7221 - precision_1

Epoch 7/10
5/8 [=================>............] - ETA: 0s - loss: 0.3365 - precision_193: 0.3500 - recall_193: 0.7000
Epoch 7: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/3/cp-0007.ckpt
8/8 [==============================] - 0s 22ms/step - loss: 0.3438 - precision_193: 0.3261 - recall_193: 0.8333 - val_loss: 0.6422 - val_precision_193: 0.0000e+00 - val_recall_193: 0.0000e+00
Epoch 8/10
6/8 [=====================>........] - ETA: 0s - loss: 0.3356 - precision_193: 0.2444 - recall_193: 0.8462
Epoch 8: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/3/cp-0008.ckpt
8/8 [==============================] - 0s 21ms/step - loss: 0.3457 - precision_193: 0.2857 - recall_193: 0.8889 - val_loss: 0.2769 - val_precision_193: 0.0000e+00 - val_recall_193: 0.0000e+00
Epoch 9/10
5/8 [=================>............] - ETA: 0s - loss: 0.2998 - preci

Epoch 5/10
5/8 [=================>............] - ETA: 0s - loss: 0.3473 - precision_195: 0.4400 - recall_195: 0.8462
Epoch 5: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/5/cp-0005.ckpt
8/8 [==============================] - 0s 25ms/step - loss: 0.3019 - precision_195: 0.3871 - recall_195: 0.8000 - val_loss: 0.4644 - val_precision_195: 0.0000e+00 - val_recall_195: 0.0000e+00
Epoch 6/10
5/8 [=================>............] - ETA: 0s - loss: 0.1863 - precision_195: 0.4000 - recall_195: 1.0000
Epoch 6: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/5/cp-0006.ckpt
8/8 [==============================] - 0s 22ms/step - loss: 0.1902 - precision_195: 0.3846 - recall_195: 1.0000 - val_loss: 0.5029 - val_precision_195: 0.0000e+00 - val_recall_195: 0.0000e+00
Epoch 7/10
5/8 [=================>............] - ETA: 0s - loss: 0.1458 - preci

Epoch 3/10
5/8 [=================>............] - ETA: 0s - loss: 0.4721 - precision_197: 0.1449 - recall_197: 1.0000
Epoch 3: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/7/cp-0003.ckpt
8/8 [==============================] - 0s 24ms/step - loss: 0.4238 - precision_197: 0.1807 - recall_197: 1.0000 - val_loss: 0.3514 - val_precision_197: 0.0000e+00 - val_recall_197: 0.0000e+00
Epoch 4/10
5/8 [=================>............] - ETA: 0s - loss: 0.4403 - precision_197: 0.5000 - recall_197: 0.5000        
Epoch 4: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/7/cp-0004.ckpt
8/8 [==============================] - 0s 22ms/step - loss: 0.4081 - precision_197: 0.3056 - recall_197: 0.7333 - val_loss: 0.7787 - val_precision_197: 0.1429 - val_recall_197: 0.6667
Epoch 5/10
6/8 [=====================>........] - ETA: 0s - loss: 0.3192 - preci

Epoch 1/10
5/8 [=================>............] - ETA: 0s - loss: 0.6682 - precision_199: 0.0000e+00 - recall_199: 0.0000e+00
Epoch 1: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/9/cp-0001.ckpt
8/8 [==============================] - 1s 55ms/step - loss: 0.7413 - precision_199: 0.0000e+00 - recall_199: 0.0000e+00 - val_loss: 0.6465 - val_precision_199: 0.1667 - val_recall_199: 1.0000
Epoch 2/10
5/8 [=================>............] - ETA: 0s - loss: 0.6284 - precision_199: 0.1169 - recall_199: 0.9000        
Epoch 2: saving model to /scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_9daysblocking/normal/random_initCNN_9/9/cp-0002.ckpt
8/8 [==============================] - 0s 23ms/step - loss: 0.6350 - precision_199: 0.1308 - recall_199: 0.8750 - val_loss: 0.6036 - val_precision_199: 0.1429 - val_recall_199: 0.5000
Epoch 3/10
6/8 [=====================>........] - ETA: 0s - loss: 0.5020